# GEOG 497 - Spring 2022 - Cryosphere & Climate Systems
## A6: Remote sensing of ice sheet surface melt

## Part 2: Experimenting with satellite-based melt detection

Input data: ASCAT backscatter and AWS-derived melt rates (see Part 1 for details).
        
For this part of the assignment, you will experiment with setting a threshold value below a mean winter backscatter value with which to classify melt vs. no-melt from the ASCAT data.

### Run the following code block first.
This loads python packages and sets some plotting-related variables. 

In [ ]:
# Import python packages

# for file searching
import glob 

# for data reading/analysis
import xarray as xr
import pandas as pd
import numpy as np

#  for geographic projections
from pyproj import Transformer

# for plotting
import matplotlib.pyplot as plt
import matplotlib.style as style
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import datetime

# seaborn adds some extra visual appeal to our plots
import seaborn as sns

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# set some universal plot settings here
plt.rcParams["figure.dpi"] = 200
plt.rcParams['axes.xmargin'] = 0.05
sns.set_style('darkgrid')
sns.set_context("notebook", font_scale=0.75)
%config InlineBackend.figure_format = 'retina' # make high res plots for retina 5k displays

# lastly, this specifies where the AWS and ASCAT data are located
awsFolder = './Data/'
ascatFolder = '/gpfs/group/ljt5282/default/ASCAT_data/netcdfs/annual/'

### Preliminary step 1: read all the ASCAT data files
This reads all of the continent-wide, daily temporal resolution ASCAT netcdf files using `xarray`. Afterward, we will subset these data to only the grid cell nearest to the AWS location of interest.

Here, we'll also read netcdfs containing winter-mean (June, July, August; JJA) backscatter grids that have been pre-calculated.

In [ ]:
# read all yearly netcdfs using xarray and create new xarray DataSet variable
ascat_ds = xr.open_mfdataset(ascatFolder + 'msfa*.nc')

# read all yearly netcdfs using xarray and create new xarray DataSet variable
ascat_jja_ds = xr.open_mfdataset(ascatFolder + 'jja_means/msfa*.nc')

### Main code: Plot ASCAT and AWS melt data at one location
Code here is all set up to work and should be used as a reference for making plots at other AWS sites.

In the block directly below, you'll want to specify what AWS datafile we'll want to read.
This then loads it using `pandas` and resamples the hourly data to daily means.

In [ ]:
# Use pandas to read SEB melt data for the AWS site
aws_datafile = 'IMAU_aws14_high-res_meteo.tab'
aws_folder_plus_file = awsFolder + aws_datafile
aws_df = pd.read_csv(aws_folder_plus_file,
                            sep='\t', 
                            skiprows=40,
                            parse_dates=['Date/Time'],
                            index_col=['Date/Time'])

# Resample hourly data to daily means and set as new pandas DataFrame variable
aws_df_daily = aws_df.resample('D').mean()

### Define the latitude and longitude of where the AWS is located. 
This info can be found in the AWS data file's header:

In [ ]:
# here we're using a shell command 'head' to read the first few lines of the file
# then we're piping it through the 'grep' command to search for text we're interested in.
# this is also a useful point to verify that we're working with the AWS number we indend to work with

!head $aws_folder_plus_file | grep LATITUDE

In [ ]:
# Using that info from the header, specify here where this AWS is located (latitude and longitude)
aws_lat = -67.020
aws_lon = -61.50

### Use pyproj to convert lat/lon coordinates in the projected x/y coordinates of the ASCAT data:

In [ ]:
# from geographic lat/lon to EPSG:3976 (WGS 84 / NSIDC Sea Ice Polar Stereographic South)
transformer = Transformer.from_crs("epsg:4326", "epsg:3976")
aws_x,aws_y = transformer.transform(aws_lat, aws_lon)

### Subset the ASCAT data to the location and timespan of the AWS data

In [ ]:
# Now, subset the ascat data to only the point (i.e., the grid cell nearest to the AWS site) and  only when it overlaps with the AWS data
ascat_ds_ts_aws = ascat_ds['sigma0'].sel(x=aws_x, method="nearest") \
                                    .sel(y=aws_y, method="nearest") \
                                    .sel(time=slice(aws_df_daily.index.min(), aws_df_daily.index.max()))

# also read the winter mean sigma0 values at the aws site 
jja_sigma0_site = ascat_jja_ds['sigma0'].sel(x=aws_x, method="nearest").sel(y=aws_y, method="nearest")

### Plot the AWS-derived melt data and ASCAT data over that site, but also now add in lines that show the winter mean backscatter at that site.

In [ ]:
# Now, plot the AWS and ASCAT data

# create a figure
fig, ax = plt.subplots(figsize=(15, 7.5))

# plot ascat data on first vetical axis
line1 = ax.plot(ascat_ds_ts_aws.time, ascat_ds_ts_aws,
                 label='ASCAT sigma0')

# plot the winter means as horizontal bars

# find the start and end years for the aws data
# then use those to make an array of start and end dates for the winter means
# lastly, convert these into datetime dates so we can plot
startyear = aws_df_daily.index.min().year-1
endyear = aws_df_daily.index.max().year+1
years = np.arange(startyear,endyear)

startdates = []
enddates = []
for i in range(len(years)):
    startdate = str(years[i]) + '-06-01'
    enddate = str(years[i]) + '-08-31'
    startdates.append(startdate)
    enddates.append(enddate)

startdates_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in startdates]
enddates_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in enddates]

# loop through winter means and plot
for i in range(len(years)):
    ax.plot([startdates_x[i],enddates_x[i]], [jja_sigma0_site.sel(time=str(years[i])).values,jja_sigma0_site.sel(time=str(years[i])).values],
            color='black',
            linewidth=3) 

# make second vertical axis; plot aws data on it
ax2 = ax.twinx()
line2 = ax2.plot(aws_df_daily['Melt rate [mm w.e.] (surface melt, within dt)'], 
                label='AWS-derived melt rate', 
                color='indianred')

# Set some display properties
ax.set(ylabel='ASCAT backscatter at AWS site [dB]',
       xlabel='Date',
       title='AWS14')

ax2.set(ylabel='Daily mean melt rate [mm w.e.]')
ax2.yaxis.grid(False)

ax2.set_ylim((-0.5, 0.5))
ax.set_ylim((-30, 30))

# add a legend now that we have 2 lines
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc='upper left')

print('Winter mean backscatter shown as horizontal black lines')
plt.show()


## Experiment with setting a melt threshold
Setting a threshold value (in dB) below the winter mean backscatter is a common way to classify the presence of meltwater at or near ice sheet surfaces due to the large decrease in backscatter that is typical with the presence of melt.

Here, experiment with setting a threshold value and see the results.

In [ ]:
# First, set a threshold in dB from the winter mean and see how it affects melt detection
threshold_value = # <- ENTER NUMBER HERE! (before #) 
melt_thresholds = jja_sigma0_site - threshold_value

In [ ]:
# This code will apply the melt detection using the above threshold_value

startdates_thresh = []
enddates_thresh = []
melt_detected_list = []
for i in range(len(years)):
    # select the ascat data in this melt year
    startdate_t = str(years[i]) + '-07-01'
    enddate_t = str(years[i]+1) + '-06-30'
    cur_melt_year_ascat = ascat_ds_ts_aws.sel(time=slice(startdate_t, enddate_t))
    # get the threshold for this year
    cur_years_threshold = melt_thresholds.sel(time=str(years[i])).values
    # set values <= than the threshold to 1, else 0
    cur_melt_classified = xr.where(cur_melt_year_ascat <= cur_years_threshold, 1, 0)
    melt_detected_list.append(cur_melt_classified) 

melt_detected_da = xr.concat(melt_detected_list, dim='time')

In [ ]:
# Lastly, this code will plot the detected melt using your specified

# create a figure
fig, ax = plt.subplots(figsize=(15, 7.5))

# plot ascat data on first vetical axis
line1 = ax.plot(ascat_ds_ts_aws.time, ascat_ds_ts_aws,
                 label='ASCAT sigma0')


# make second vertical axis; plot aws data on it
ax2 = ax.twinx()
line2 = ax2.plot(aws_df_daily['Melt rate [mm w.e.] (surface melt, within dt)'], 
                label='AWS-derived melt rate', 
                color='indianred')

# plot the winter means as horizontal bars

# find the start and end years for the aws data
# then use those to make an array of start and end dates for the winter means
# lastly, convert these into datetime dates so we can plot
startyear = aws_df_daily.index.min().year-1
endyear = aws_df_daily.index.max().year+1
years = np.arange(startyear,endyear)

startdates = []
enddates = []
for i in range(len(years)):
    startdate = str(years[i]) + '-06-01'
    enddate = str(years[i]) + '-08-31'
    startdates.append(startdate)
    enddates.append(enddate)

startdates_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in startdates]
enddates_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in enddates]

# loop through winter means and plot
for i in range(len(years)):
    ax.plot([startdates_x[i],enddates_x[i]], [jja_sigma0_site.sel(time=str(years[i])).values,jja_sigma0_site.sel(time=str(years[i])).values],
            color='black',
            linewidth=3) 

# now move on to the same logic for the melt thresholds
startdates_thresh = []
enddates_thresh = []
for i in range(len(years)):
    startdate_t = str(years[i]) + '-07-01'
    enddate_t = str(years[i]+1) + '-06-30'
    startdates_thresh.append(startdate_t)
    enddates_thresh.append(enddate_t)

startdates_thresh_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in startdates_thresh]
enddates_thresh_x = [datetime.datetime.strptime(d,"%Y-%m-%d").date() for d in enddates_thresh]

for i in range(len(years)):
    ax.plot([startdates_thresh_x[i],enddates_thresh_x[i]], 
            [melt_thresholds.sel(time=str(years[i])).values,melt_thresholds.sel(time=str(years[i])).values],
            color='dimgray',
            linewidth=2,
            linestyle='dotted') 

# show detected melt
ax3 = ax.twinx()
line3 = ax3.fill_between(melt_detected_da.time.values,melt_detected_da, 
                label='ASCAT-detected melt', 
                color='cornflowerblue',
                alpha=0.3)

# Set some display properties
ax.set(ylabel='ASCAT backscatter at AWS site [dB]',
       xlabel='Date',
       title='AWS14')

ax2.set(ylabel='Daily mean melt rate [mm w.e.]')
ax2.yaxis.grid(False)

ax3.set_ylim((0,1))
ax2.set_ylim((-0.5, 0.5))
ax.set_ylim((-30, 30))

ax3.yaxis.grid(False)

# add a legend now that we have 2 lines
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax.legend(lines, labels, loc='lower left')

print('Winter mean backscatter shown as horizontal black lines')
print('Melt thresholds shown as horizontal dotted lines')
print('Periods of detected melt shown as vertical blue blocks')

plt.show()

### What do you think?
Does your `melt_threshold` seem to capture the periods of melt? You may want to experiment using a few different values to see the effect on melt detection. Once satisfied, move on to the next step in the assignment.